In [2]:
import json
import requests
import pandas as pd
import numpy as np


# input ORCID ID for Jack Brookshire
orcid = 'https://orcid.org/0000-0002-0412-7696'

#build API URL for one author works
def build_author_works_url(orcid):
    # specify endpoint
    endpoint = 'works'

    # build the 'filter' parameter
    filters = (
      f'author.orcid:{orcid}',
      'is_paratext:false'
    )

    # put the URL together
    return f'https://api.openalex.org/{endpoint}?filter={",".join(filters)}'

author_works_url = build_author_works_url(orcid)

#get JSON data from API URL and put it in the variable "response_data"
r = requests.get(author_works_url)
response_data = r.json()

related_results = [result['related_works'] for result in response_data['results']]

related_strip_to_W = np.char.strip(related_results, chars ='https://openalex.org/')

article_results = np.ravel(related_strip_to_W)

article_results_list = article_results.tolist()

#article_url = "https://api.openalex.org/works/"

for x in article_results_list:
    article_url = [f'https://api.openalex.org/works/' + x for x in article_results_list]

i = 0
s = len(article_url)
output_results = []
while i < s:
    x = requests.get(article_url[i])
    article_data = x.json()
    article_year = article_data.get('publication_year')
    if article_year >= 2023:
        result_filtered = output_results.append(article_data)
    else:
        pass
    i = i+1

output_to_dataframe = pd.DataFrame.from_dict(output_results)
cleanup = output_to_dataframe.get(['doi','title','publication_date','authorships'])
final = cleanup.to_csv('final_recommendations.csv', encoding='utf-8')
